In [1]:
# import libraries
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix, vstack

from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
song_df= pd.read_pickle('../../data_lyrics/pd/fx_th_songs_35.pkl')

In [101]:
song_df.to_csv('temp.csv')

In [3]:
song_df.head()

,artist,song_name,href,lyric,lines,words,n_words,words_str,artists,duplicates,n_artists,n_unique_words,unique_words_ratio,words_per_line,tf_idf_vector,tf_idf_score
0,bird_thongchai,Okay,/music/thailyric/13588,ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...,57,"[ไม่, ว่า, จะ, เป็น, ยังไง, babi, it, okay, ไม...",343,ไม่ ว่า จะ เป็น ยังไง babi it okay ไม่ ว่า จะ ...,{bird_thongchai},0,1,134,0.390671,6.017544,"(0, 14986)\t0.02706318124303119\n (0, 14978...",6.314019
1,bird_thongchai,กว่าจักรวาล,/music/thailyric/13978,จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...,31,"[จะ, ยอม, นั่ง, จรวด, ไป, ตรวจ, ดาวอังคาร, หาก...",239,จะ ยอม นั่ง จรวด ไป ตรวจ ดาวอังคาร หาก ว่าที่ ...,{bird_thongchai},0,1,70,0.292887,7.709677,"(0, 14973)\t0.07331503970888636\n (0, 14914...",6.755620
2,bird_thongchai,กำแพง,/music/thailyric/14111,ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...,39,"[ถ้า, เคย, พบ, เจอ, กำแพง, ที่, ดู, ทั้ง, ใหญ่...",316,ถ้า เคย พบ เจอ กำแพง ที่ ดู ทั้ง ใหญ่ และ สูงช...,{bird_thongchai},0,1,134,0.424051,8.102564,"(0, 14978)\t0.0895602203600452\n (0, 14973)...",9.597515
3,bird_thongchai,ชีวิตเดี่ยว,/music/thailyric/13796,อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...,50,"[อยู่, ตรงนี้, แค่, เพียงลำพัง, กับ, ความเหงา,...",250,อยู่ ตรงนี้ แค่ เพียงลำพัง กับ ความเหงา ที่ เป...,{bird_thongchai},0,1,91,0.364000,5.000000,"(0, 14978)\t0.08974985827768892\n (0, 14973...",7.788788
4,bird_thongchai,ผู้ต้องหา,/music/thailyric/13825,แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...,65,"[แค่, ตัว, คนเดียว, ไม่, ตาย, ล่ะ, มั้ง, ถาม, ...",416,แค่ ตัว คนเดียว ไม่ ตาย ล่ะ มั้ง ถาม ใจ กี่ คร...,{bird_thongchai},0,1,96,0.230769,6.400000,"(0, 14973)\t0.033155545848114705\n (0, 1488...",6.089520


In [4]:
def select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max):
    song_count_df = song_df.groupby('artist')[['artist']].count().rename(columns={'artist': 'count'})
    artist_lst = list(song_count_df.loc[song_count_df['count'] >= n_song_min].index.values)

    n_set_total = sum(n_set.values())

    artist_set = []
    while len(artist_set) < n_set_total:
        new_artist = tuple(np.random.choice(artist_lst, size=n_artist, replace=False))
        if new_artist not in artist_set:
            artist_set.append(new_artist)

    # split artist sets
    artist_select = {}
    for field, n in n_set.items():
        i_select = np.random.choice(range(len(artist_set)), size=n, replace=False)
        artist_list = list(artist_set)
        artist_select[field] = [artist_list[i] for i in i_select]
        artist_set = [s for s in artist_set if s not in artist_select[field]]
    # create dataframe with all features
    feature_dict = {}
    # dictionary to map artist set id to list of artists
    set_id_to_artist_tp = {}

    i = 0
    for field, artist_set in artist_select.items():
        df_lst = []
        for artist_tp in artist_set:
            i += 1
            df = song_df.loc[song_df['artist'].isin(artist_tp), 
                             ['artist', 'song_name', 'n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_vector', 'tf_idf_score']]
            # check if number of songs is too high
            if len(df) * n_artist > n_song_artist_max:
                df = df.sample(int(n_song_artist_max / n_artist), random_state=0)
                
            df['artist_set_id'] = i
            set_id_to_artist_tp[i] = artist_tp
            df_lst.append(df)
        feature_dict[field] = pd.concat(df_lst)  
        print('Number of songs in {}: {}'.format(field, len(feature_dict[field])))

    # get all selected artists
    artist_select_set = set.union(*[set(sum(tp_lst, ())) for tp_lst in artist_select.values()])

    # calculate mean vector
    def get_mean_vector(vec_lst):
        return csr_matrix(vstack(vec_lst).mean(axis=0))

    # create artist dataframe from training data
    df_lst = []
    for artist, df in song_df.loc[song_df['artist'].isin(artist_select_set)].groupby('artist'):
        dic = {'artist': artist}
        # calculate averages and standard diviations
        for field in ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score']:
            dic[field + '_mean'] = df[field].mean()
            dic[field + '_std'] = df[field].std()

        # number of songs
        dic['songs'] = len(df)

        # calculate average tf idf vector
        dic['tf_idf_vector_mean'] = get_mean_vector(df['tf_idf_vector'])

        df_lst.append(pd.DataFrame(dic, index=[0]))
    artist_feature_df = pd.concat(df_lst)

    def get_features(df):
        # get artist set id
        artist_set_id = df['artist_set_id'].iloc[0]
        
        # get all artists
        artist_feature_select_df = artist_feature_df.loc[artist_feature_df['artist']\
                                                         .isin(set_id_to_artist_tp[artist_set_id])]

        # merge dataframes
        artist_song_feature_df = pd.merge(artist_feature_select_df.assign(key=0), df.assign(key=0), on='key', 
                                          suffixes=['_artist', '_song']).drop('key', axis=1)    
        artist_song_feature_df['same_artist'] = \
            artist_song_feature_df['artist_artist'] == artist_song_feature_df['artist_song']

        # calculate features
        # add feature polarity
        for feature in ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score']:
            artist_song_feature_df[feature + '_diff'] = \
                artist_song_feature_df[feature] - artist_song_feature_df[feature + '_mean']
            artist_song_feature_df[feature + '_diff_std'] = \
                artist_song_feature_df[feature + '_diff'] / artist_song_feature_df[feature + '_std']
        
        # calculate similarity of artist tf idf vector and song vector
        def tf_idf_vector_similarity(artist_vector, song_vector, songs, same_artist):
            # check if song is from same artist
            if same_artist:
                # deduct song vector from artist vector
                artist_vector = (songs * artist_vector - song_vector) / (songs - 1)
            # calculate similarity
            return cosine_similarity(artist_vector, song_vector)[0][0]

        # calculate vector similarity between artist and song
        artist_song_feature_df['vector_similarity'] = \
            artist_song_feature_df.apply(lambda row: tf_idf_vector_similarity(row['tf_idf_vector_mean'], 
                                                      row['tf_idf_vector'], row['songs'], row['same_artist']), axis=1)    
        return artist_song_feature_df

    artist_song_feature = {}
    for field in feature_dict:
        artist_song_feature[field] = feature_dict[field].groupby('artist_set_id').apply(get_features)\
                                                        .reset_index(drop=True)
        
    return artist_song_feature

In [68]:
feature = ['n_words_diff', 'n_words_diff_std',
        'unique_words_ratio_diff', 'unique_words_ratio_diff_std',
        'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff',
        'tf_idf_score_diff_std','vector_similarity']
df_lst = []

pd.options.mode.chained_assignment = None  # default='warn'

In [80]:
def prepare_data(df, feature_org, feature_abs):
    for f in feature_abs:
        df[f] = df[f].abs()
    X = df[feature_org + feature_abs].values
    y = df['same_artist'].values
    
    return X, y

def select_songs_train_model(song_df, n_set, n_artist, n_song_min, n_song_artist_max, feature_org, feature_abs, model):
    artist_song_feature = select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max)

    # prepare data
    X, y = prepare_data(artist_song_feature['train'], feature_org, feature_abs)

    model = model.fit(X, y)
    
    return artist_song_feature, model

In [81]:
from sklearn.naive_bayes import GaussianNB

In [83]:
feature_org = ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score', 'vector_similarity']
feature_abs = ['n_words_diff', 'n_words_diff_std', 'unique_words_ratio_diff', 'unique_words_ratio_diff_std', 
               'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std']
# prepare data create and train pipeline
n_artist = 3
n_song_min = 5
n_set = {'train': 100}
n_song_artist_max = 100

feature_org = ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score', 'vector_similarity']
feature_abs = ['n_words_diff', 'n_words_diff_std', 'unique_words_ratio_diff', 'unique_words_ratio_diff_std', 
               'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std']

model = GaussianNB()

np.random.seed(1)
artist_song_feature, pipeline = select_songs_train_model(song_df, n_set, n_artist, n_song_min, n_song_artist_max, 
                                                            feature_org, feature_abs, model)

Number of songs in train: 3300


In [84]:
def predict_artist(df, feature_org, feature_abs, model, top_n):
    # prepare data
    X, y = prepare_data(df, feature_org, feature_abs)
    
    # get probability
    proba = model.predict_proba(X)
    # attach to dataframe
    df['probability'] = proba[:, 1]
    df['correct_prediction'] = df['artist_artist'] == df['artist_song']
    
    # get artist song pairs with highest probability
    predict_select = df.sort_values('probability', ascending=False).groupby(['artist_set_id']).head(top_n)\
                       .groupby(['artist_set_id'])['correct_prediction'].max()
    
    # print(predict_select)
    # get accuracy
    print('Accuracy: {}'.format(predict_select.mean()))
    
    return predict_select

In [92]:
n_artist_lst = [2, 4, 8, 16, 32]
top_n_lst = [1, 2, 4, 8, 16]
n_song_artist_max = 128
np.random.seed(2)

n_set = {'train': 100, 'val': 100}

feature_org = ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score', 'vector_similarity']
feature_abs = ['n_words_diff', 'n_words_diff_std', 'unique_words_ratio_diff', 'unique_words_ratio_diff_std', 
               'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std', ]

result_lst = []

for n_artist in n_artist_lst:
    print(datetime.now())
    print('n_artist: {}'.format(n_artist))
    
    artist_song_feature, model = select_songs_train_model(song_df, n_set, n_artist, n_song_min, 
                                                                n_song_artist_max, feature_org, feature_abs, model)
    
    for top_n in [n for n in top_n_lst if n < n_artist]:
        print('top_n: {}'.format(top_n))
        
        predict_select = predict_artist(artist_song_feature['val'], feature_org, feature_abs, model, top_n=top_n)
        
        result_dict = {'n_artist': n_artist, 'top_n': top_n, 'accuracy': predict_select.mean()}
        result_lst.append(result_dict)
        
    print('')
    
result_df = pd.DataFrame(result_lst)

2022-05-26 23:07:16.650462
n_artist: 2
Number of songs in train: 6400
Number of songs in val: 6400
top_n: 1
Accuracy: 0.81

2022-05-26 23:07:24.119629
n_artist: 4
Number of songs in train: 3200
Number of songs in val: 3200
top_n: 1
Accuracy: 0.45
top_n: 2
Accuracy: 0.78

2022-05-26 23:07:30.625977
n_artist: 8
Number of songs in train: 1600
Number of songs in val: 1600
top_n: 1
Accuracy: 0.34
top_n: 2
Accuracy: 0.55
top_n: 4
Accuracy: 0.77

2022-05-26 23:07:36.767563
n_artist: 16
Number of songs in train: 800
Number of songs in val: 800
top_n: 1
Accuracy: 0.5
top_n: 2
Accuracy: 0.63
top_n: 4
Accuracy: 0.73
top_n: 8
Accuracy: 0.85

2022-05-26 23:07:42.707381
n_artist: 32
Number of songs in train: 400
Number of songs in val: 400
top_n: 1
Accuracy: 0.07
top_n: 2
Accuracy: 0.16
top_n: 4
Accuracy: 0.28
top_n: 8
Accuracy: 0.5
top_n: 16
Accuracy: 0.78

2022-05-26 23:07:48.639053
n_artist: 64
Number of songs in train: 200
Number of songs in val: 200
top_n: 1
Accuracy: 1.0
top_n: 2
Accuracy: 1.0